**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

# CELINE YOUSSEF MVA MP3

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import skvideo
skvideo.setFFmpegPath("/usr/local/bin/")

In [4]:
import skvideo.io
import cv2
import json
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [5]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [6]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=False):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
# __Question 1__:
Explain the function act. Why is ```epsilon``` essential?

- The function act is a function used to provide the next action (between 0 and 4: for 4 different actions) for a given state and uses a learned policy. More precisely an epsilon greedy behavioral policy. 




- It is essential to have an epsilon value in order for the agent to learn more policies and be able to reach an optimal solution without being stuck. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [7]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[:2,:]= -1
        self.position[:,:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        
        self.board[self.position < 0] = 0
        self.board[self.x,self.y] = 0
        
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [8]:
# parameters
size = 13
T= 200
temperature=0.3
epochs_train= 300 # set small when debugging
epochs_test= 50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

# __Question 2__ 
Explain the use of the arrays ```position``` and ```board```.

- The arrays position are made for storing the current state of the agent in a game and provide information about what positions are possible and not possible to go to (ex: borders).

- The arrays board are made for storing information about states where we have a cheese or poison or nothing.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [9]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        #pass
        return np.random.randint(0, self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [10]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(1, epochs + 1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix + str(e))

        # Update stats
        score += win - lose

        print("Win/lose count {}/{}. Average score ({})".format(win, lose, score / e))
    print('Final score: {}'.format(score / epochs))

In [108]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=0.3)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix='random')
HTML(display_videos('random1.mp4'))

Win/lose count 6.0/9.0. Average score (-3.0)
Win/lose count 11.0/19.0. Average score (-5.5)
Win/lose count 9.5/10.0. Average score (-3.8333333333333335)
Win/lose count 11.0/15.0. Average score (-3.875)
Win/lose count 10.5/15.0. Average score (-4.0)
Win/lose count 9.5/15.0. Average score (-4.25)
Win/lose count 12.0/10.0. Average score (-3.357142857142857)
Win/lose count 5.5/15.0. Average score (-4.125)
Win/lose count 3.5/7.0. Average score (-4.055555555555555)
Win/lose count 9.0/17.0. Average score (-4.45)
Win/lose count 17.0/14.0. Average score (-3.772727272727273)
Win/lose count 13.5/11.0. Average score (-3.25)
Win/lose count 10.0/14.0. Average score (-3.3076923076923075)
Win/lose count 7.0/17.0. Average score (-3.7857142857142856)
Win/lose count 11.0/20.0. Average score (-4.133333333333334)
Win/lose count 10.0/11.0. Average score (-3.9375)
Win/lose count 7.0/13.0. Average score (-4.0588235294117645)
Win/lose count 11.0/19.0. Average score (-4.277777777777778)
Win/lose count 8.0/14.0.

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}





1) 

The Q-function is known as: $Q^\pi(s,a)=E_{{\pi}}[\sum_{t=0}^\infty\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]$. 

Then we get the Bellman's equation by putting aside the first term: $Q^\pi(s,a)=E_{p^{\pi}}[r(s, a) + \gamma \sum_{t=1}^\infty\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]$. 

We can represent the state and action in term of $p^\pi(.|s, a)$, so we have:  $Q^\pi(s,a)= r(s, a) + \gamma E_{(s^\prime, a^\prime)\sim p^\pi(.|s,a)}[Q^\pi(s^\prime, a^\prime)]$.


2)

The optimal Q-function is known as: $Q^*(s,a)=\max_{\pi}Q^\pi(s,a)$.

This function can be maximized with respect to the policy : $\pi$, which is the same as optimizing the function with respect to a' from the previous equation in part 1  : $Q^*(s,a)= r(s,a) + \gamma E_{(s^\prime, a^\prime)\sim p^{\pi^*}(.|s,a)} [\max_{a^\prime}Q^*(s^\prime,a^\prime)]$.

3)
Plausible objective:

The goal is to find the optimal Q-function by minimizing the distance between the first two equations (prediction and target): $L^2$-error.

So we have: \begin{equation}0 =E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')] - Q^{}(s,a) \end{equation} \begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}



***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [52]:
class Memory(object):
    def __init__(self, max_memory=100):  #define the max_memory to be equal to 100
        self.max_memory = max_memory     #initialize max_memory
        self.memory = list()            #store the moves in a list - buffer

    def remember(self, m):
        #pass
        self.memory.append(m)
        self.memory = self.memory[:self.max_memory]

    def random_access(self):
        #pass
        return self.memory[np.random.randint(0, len(self.memory))]

***
The pipeline we will use for training is given below:

In [53]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [111]:
from keras.layers import Input, Flatten
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None]))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            if game_over_:
                Q = np.zeros(4) #end state
            else:
                Q = self.model.predict(n_s_[None])[0]
                target_q[i][a_] = r_ + self.discount * np.max(Q) 
        target_q = np.clip(target_q, -3, 3)  #clip target

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        model = Sequential([
            Dense(32, input_shape = (5, 5, 2)),
            Flatten(input_shape=(5, 5, self.n_state)),
            Dense(32, activation='relu'),
            Dense(4, activation = 'linear')
        ])
        

        model.summary()
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [112]:
size = 13
T = 200
epochs_train = 20
env = Environment(grid_size=size, max_time=T, temperature=0.5)
agent = DQN_FC(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_87 (Dense)             (None, 5, 5, 32)          96        
_________________________________________________________________
flatten_53 (Flatten)         (None, 800)               0         
_________________________________________________________________
dense_88 (Dense)             (None, 32)                25632     
_________________________________________________________________
dense_89 (Dense)             (None, 4)                 132       
Total params: 25,860
Trainable params: 25,860
Non-trainable params: 0
_________________________________________________________________
Epoch 000/020 | Loss 0.0000 | Win/lose count 0.5/1.0 (-0.5)
Epoch 001/020 | Loss 0.0000 | Win/lose count 1.5/0 (1.5)
Epoch 002/020 | Loss 0.0007 | Win/lose count 0/2.0 (-2.0)
Epoch 003/020 | Loss 0.0001 | Win/lose count 0.5/0 (0.5)
Epoch 004/020 | Loss 0.0025 | Win/lose

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [121]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        """
        model = Sequential()
        model.add(Conv2D(16, (3, 3), input_shape = (5, 5, 2), activation = 'relu')) 
        model.add(Conv2D(8, (2, 2), activation = 'relu'))
        model.add(Flatten())
        model.add(Dense(4, activation = 'relu'))
        """
     
        model = Sequential()
        model.add(Conv2D(64, (2,2), input_shape = (5, 5, 2)))
        model.add(Conv2D(32, (2,2)))
        model.add(Flatten())
        model.add(Dense(32, activation ='relu'))
        model.add(Dense(4, activation = 'relu'))
        

        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [122]:
size = 13
T = 200
epochs_train = 20
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/020 | Loss 0.0000 | Win/lose count 0.5/3.0 (-2.5)
Epoch 001/020 | Loss 0.0015 | Win/lose count 1.5/3.0 (-1.5)
Epoch 002/020 | Loss 0.0001 | Win/lose count 2.0/1.0 (1.0)
Epoch 003/020 | Loss 0.0017 | Win/lose count 1.0/0 (1.0)
Epoch 004/020 | Loss 0.0000 | Win/lose count 0.5/0 (0.5)
Epoch 005/020 | Loss 0.0003 | Win/lose count 1.0/1.0 (0.0)
Epoch 006/020 | Loss 0.0067 | Win/lose count 0.5/1.0 (-0.5)
Epoch 007/020 | Loss 0.0001 | Win/lose count 1.0/0 (1.0)
Epoch 008/020 | Loss 0.0000 | Win/lose count 2.5/2.0 (0.5)
Epoch 009/020 | Loss 0.0000 | Win/lose count 4.5/1.0 (3.5)
Epoch 010/020 | Loss 0.0022 | Win/lose count 2.0/0 (2.0)
Epoch 011/020 | Loss 0.0000 | Win/lose count 2.0/2.0 (0.0)
Epoch 012/020 | Loss 0.0043 | Win/lose count 2.5/2.0 (0.5)
Epoch 013/020 | Loss 0.0000 | Win/lose count 2.5/2.0 (0.5)
Epoch 014/020 | Loss 0.0000 | Win/lose count 1.0/0 (1.0)
Epoch 015/020 | Loss 0.0000 | Win/lose count 2.0/1.0 (1.0)
Epoch 016/020 | Loss 0.0002 | Win/lose count 2.0/6.0 (-4.0)
Epo

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

- Observations: 

We can see from the testing part that DQN with CNN gives better performance than with FC. 

Regarding the change of temperature, when it increased from 0.3 to 0.5, the testing results were better for CNN and worse for FC

Mainly both scores increase with the temperature because there are more rewards, but CNN is more adaptable. And the gap between CNN and FC becomes wider when the temperature increases.

- Issues:

The agent is not totally exploring its environment and tend to be stuck between two positions when it does not see anymore reward in its surrounding. This can explain the reason for low scores.

### Temperature = 0.3

In [123]:
epochs_test = 20
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_103 (Dense)            (None, 5, 5, 32)          96        
_________________________________________________________________
flatten_61 (Flatten)         (None, 800)               0         
_________________________________________________________________
dense_104 (Dense)            (None, 32)                25632     
_________________________________________________________________
dense_105 (Dense)            (None, 4)                 132       
Total params: 25,860
Trainable params: 25,860
Non-trainable params: 0
_________________________________________________________________
Test of the CNN
Win/lose count 3.5/0. Average score (3.5)
Win/lose count 0.5/0. Average score (2.0)
Win/lose count 2.5/1.0. Average score (1.8333333333333333)
Win/lose count 1.0/0. Average score (1.625)
Win/lose count 3.0/0. Average score (1.9)
Win/lose count 3.0/0. Ave

### Temperature = 0.5

In [124]:
epochs_test = 20
env = Environment(grid_size=size, max_time=T,temperature=0.5)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_108 (Dense)            (None, 5, 5, 32)          96        
_________________________________________________________________
flatten_63 (Flatten)         (None, 800)               0         
_________________________________________________________________
dense_109 (Dense)            (None, 32)                25632     
_________________________________________________________________
dense_110 (Dense)            (None, 4)                 132       
Total params: 25,860
Trainable params: 25,860
Non-trainable params: 0
_________________________________________________________________
Test of the CNN
Win/lose count 2.0/0. Average score (2.0)
Win/lose count 5.0/1.0. Average score (3.0)
Win/lose count 8.5/1.0. Average score (4.5)
Win/lose count 1.5/0. Average score (3.75)
Win/lose count 4.0/0. Average score (3.8)
Win/lose count 2.5/0. Average score (3.

In [125]:
HTML(display_videos('cnn_test10.mp4'))

In [126]:
HTML(display_videos('fc_test10.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [127]:
def train_explore(agent,env,epoch,prefix=''):
    
    #USE SAME TRAINING AGENT FROM BEFORE BUT WITH SMALL MODIFICATION RELATED TO EPSILON
    score = 0
    loss = 0
    discount=0.99 #added discount
    epsilon= agent.epsilon
    
    for e in range(epoch):

        state = env.reset()
        game_over = False

        win = 0
        lose = 0

        agent.set_epsilon(epsilon*discount) #SET E = EPSILON * DISCOUNT EPSILON
        
        while not game_over:

            action = agent.act(state)

            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        if e % 10 == 0:
            env.draw(prefix+str(e))

        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')   

        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        Environment.__init__(self, grid_size, max_time, temperature)  # constructor initiliaze
        self.malus_position = np.zeros((self.grid_size,self.grid_size)) #  penalize state
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b
    
    def act(self, action, train=False):
        """This function gets the state, reward, and game over state
        + penalizes if being on visited cells then returns the new state
        + check if end of game """

        state, reward, game_over = Environment.act(self,action)

        if train:
            reward = reward-self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1  #set a malus equal to -0.1 to an already visited cell

        reward = reward + self.board[self.x, self.y]
            
        cell_visited= self.malus_position.reshape(self.grid_size, self.grid_size,1)[self.x-2:self.x+3,self.y-2:self.y+3,:]
        state = np.concatenate((cell_visited, state),axis=2)

        return state, reward, game_over
    
    def reset(self):
        """Reset function """
        state = Environment.reset(self) 
        
        self.malus_position = np.zeros((self.grid_size,self.grid_size)) #initiaize malus pos
        self.malus_position[-2:,:]=1  #attribute 1 to border cells
        self.malus_position[:,-2:]=1
        self.malus_position[:2,:]=1
        self.malus_position[:,:2]=1
        self.malus_position[self.x, self.y] = 0.1
        
        cell_visited = self.malus_position.reshape(self.grid_size, self.grid_size,1)[self.x-2:self.x+3,self.y-2:self.y+3,:]
        state = np.concatenate((cell_visited, state),axis=2)

        return state

In [128]:
from keras.layers import Input, Flatten
# modify the CNN to match it with the new number of states and improve it

class DQN_CNN2(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN2, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(32, (2,2), input_shape = (5, 5, self.n_state)))
        model.add(Conv2D(32, (2,2)))
        model.add(Flatten())
        model.add(Dense(32, activation ='relu'))
        model.add(Dense(4, activation = 'linear'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        print(self.model.summary())

In [129]:
# Training
size = 13
T = 200
epochs_train= 20
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.5)
agent = DQN_CNN2(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 16,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_79 (Conv2D)           (None, 4, 4, 32)          416       
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 3, 3, 32)          4128      
_________________________________________________________________
flatten_64 (Flatten)         (None, 288)               0         
_________________________________________________________________
dense_111 (Dense)            (None, 32)                9248      
_________________________________________________________________
dense_112 (Dense)            (None, 4)                 132       
Total params: 13,924
Trainable params: 13,924
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/020 | Loss 0.0052 | Win/lose count 4.5/21.100000000000023 (-16.600000000000023)
Epoch 001/020 | Loss 0.0149 | Win/lose coun

In [130]:
# Evaluation

size = 13
T = 200
temperature = 0.5
epochs_test=20
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 2.5/0. Average score (2.5)
Win/lose count 9.0/1.0. Average score (5.25)
Win/lose count 9.5/2.0. Average score (6.0)
Win/lose count 3.0/0. Average score (5.25)
Win/lose count 5.5/0. Average score (5.3)
Win/lose count 7.5/0. Average score (5.666666666666667)
Win/lose count 5.5/1.0. Average score (5.5)
Win/lose count 5.5/0. Average score (5.5)
Win/lose count 9.0/0. Average score (5.888888888888889)
Win/lose count 10.0/1.0. Average score (6.2)
Win/lose count 7.0/2.0. Average score (6.090909090909091)
Win/lose count 1.0/0. Average score (5.666666666666667)
Win/lose count 5.0/1.0. Average score (5.538461538461538)
Win/lose count 1.5/0. Average score (5.25)
Win/lose count 11.0/1.0. Average score (5.566666666666666)
Win/lose count 2.0/0. Average score (5.34375)
Win/lose count 8.0/1.0. Average score (5.4411764705882355)
Win/lose count 3.5/1.0. Average score (5.277777777777778)
Win/lose count 11.0/1.0. Average score (5.526315789473684)
Win/lose count 2.5/2.0. Average score (5.275)

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***